In [1]:
from subprocess import Popen, PIPE, CalledProcessError
import re
import platform
import shutil
import os.path
import fileinput
import ipywidgets as widgets
from io import BytesIO
import xml.etree.cElementTree as ET
from functools import partial

options = {'stdout': PIPE, 'stderr': PIPE, 'bufsize' : 1, 'universal_newlines' : True, 'shell' : False}
if (platform.system() == 'Windows'):
    options['shell'] = True
    #Packages required to generate ssh keys in windows
    from cryptography.hazmat.primitives import serialization as crypto_serialization
    from cryptography.hazmat.primitives.asymmetric import rsa
    from cryptography.hazmat.backends import default_backend as crypto_default_backend

def callPopen(cmd):
    print(cmd)
    with Popen(cmd.split(),**options) as p:
        for line in p.stdout:
            print(line, end='')
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)

def isInstance(name):
    instanceExists=False
    ip=''
    with Popen('gcloud compute instances list'.split(),**options) as p:
        for line in p.stdout:
            if re.match('^{}'.format(name), line):
                instanceExists=True
                ip = line.strip().split()
                ip = ip[4]
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)
        return(instanceExists,ip)
    
            
def text_prepender(filename, text):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(text.rstrip('\r\n') + content)

def replace(file,pattern,replace):
    fileinput.close()
    for line in fileinput.input(file, inplace=True):
        print( re.sub(pattern,
                      replace,
                      line.rstrip()
                      ) 
             )
        
if (platform.system() == 'Windows'):
    def generateSSHKey(username,savePath):
        key = rsa.generate_private_key(
            backend=crypto_default_backend(),
            public_exponent=65537,
            key_size=2048
            )
        private_key = key.private_bytes(
            crypto_serialization.Encoding.PEM,
            crypto_serialization.PrivateFormat.TraditionalOpenSSL,
            crypto_serialization.NoEncryption()
            )
        public_key = key.public_key().public_bytes(
            crypto_serialization.Encoding.OpenSSH,
            crypto_serialization.PublicFormat.OpenSSH
            )
        public_file = os.path.join(savePath,username + '.pub')
        private_file = os.path.join(savePath,username)
        text_file = open(public_file, "w")
        text_file.write(public_key.decode('utf-8') + ' ' + username)
        text_file.close()
        text_file = open(private_file, "w")
        text_file.write(private_key.decode('utf-8'))
        text_file.close()
        print('Successfully created key pair')
            
if (platform.system() == 'Linux'):
    def generateSSHKey(username,savePath):
        p = Popen("echo 'yes' | ssh-keygen -t rsa -f {0}/{1} -C {1} -N '' ".format(savePath,username),
              stdout=PIPE,
              shell=True,
              stderr=PIPE
               )
        print(p.communicate())    

# Automated calibration for the GOTM/FABM model at Langtjern

## Pre-requisites

* gcloud command line utility
* google cloud account, you might need to be an admin for some of this to work, also this is not free
* ssh key setup in gitlab.au.dk
* fabric3 needs to be (pip) installed
* python modules listed in the first cell of this notebook

## Creating an instance on Google Cloud

In [2]:
instanceName = 'gotm-langtjern'
username = 'jose-luis'

createInstance = '''\
gcloud compute instances create {} \
--zone europe-west3-a \
--image-family debian-9 \
--image-project debian-cloud \
--machine-type n1-standard-2 \
'''
#--boot-disk-size 200GB \
#--machine-type n1-highcpu-32  n1-standard-2\
deleteInstance = '''\
gcloud compute instances delete {} \
--zone europe-west-3a \
'''

listInstances = '''gcloud compute instances list'''

addSSHKeys = '''gcloud compute instances add-metadata {} --zone europe-west3-a --metadata-from-file ssh-keys={}'''

if (platform.system() == 'Linux'):
    keyDir = ('/home/jose-luis/.ssh/gotmKeys')

if (platform.system() == 'Windows'):
    keyDir = ('c:/Users/jose_luis_guerrero/gotmKeys')
    

ip=''
instanceExists,ip = isInstance(instanceName)

if instanceExists:
    print('Instance {} is {}'.format(instanceName,ip) )

isStarted = False
if instanceExists and ip == 'TERMINATED' :
    callPopen('gcloud compute instances start {} --zone europe-west3-a'.format(instanceName))
    instanceExists,ip = isInstance(instanceName)
    isStarted = True
    print("Machine started and ip is {}".format(ip))

wasCreated=False
if not instanceExists and not isStarted:
    callPopen(createInstance.format(instanceName))
    wasCreated=True
    if os.path.exists(keyDir):
        shutil.rmtree(keyDir)
    os.mkdir(keyDir)
    generateSSHKey(username,keyDir)
    keyFile = os.path.join(keyDir,username + '.pub')
    text_prepender('{}/{}.pub'.format(keyDir,username), '{}:'.format(username) )
    callPopen(addSSHKeys.format(instanceName,keyDir + '/{}.pub'.format(username)))
    #callPopen('sed -i s/^{0}:// {1}/{0}.pub'.format(username,keyDir))
    replace(keyFile,"^{}:".format(username),"")
    ip=isInstance(instanceName)[1]
        #callPopen('chmod 400 {}'.format(keyDir))
        
print("The ip of {} is {}".format(instanceName,ip))

gcloud compute instances create gotm-langtjern --zone europe-west3-a --image-family debian-9 --image-project debian-cloud --machine-type n1-standard-2 
NAME            ZONE            MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
gotm-langtjern  europe-west3-a  n1-standard-2               10.156.0.6   35.198.75.197  RUNNING
Created [https://www.googleapis.com/compute/v1/projects/nivacatchment/zones/europe-west3-a/instances/gotm-langtjern].
(b"Generating public/private rsa key pair.\nYour identification has been saved in /home/jose-luis/.ssh/gotmKeys/jose-luis.\nYour public key has been saved in /home/jose-luis/.ssh/gotmKeys/jose-luis.pub.\nThe key fingerprint is:\nSHA256:M6WgpiZwCHfsEw0NpQ8ByfZ9gDmclaOhKUav1Tw+hi8 jose-luis\nThe key's randomart image is:\n+---[RSA 2048]----+\n| ..++Oo.         |\n| .+.B==          |\n|o.o+O=oo  .      |\n|o+o=oO...o       |\n|+.+ B o.S        |\n|.o + =   o       |\n|. o o .          |\n| o E .           |\n|    .            |\n+----[S

## Updating fabfile.py with credentials and ip

In [3]:
if (platform.system() == "Linux"):
    callPopen("sed -i s/^env\.hosts.*/env.hosts=\['{}']/ fabfile.py".format(ip))
    callPopen("sed -i s/^env\.user.*/env.user=\'{}\'/ fabfile.py".format(username))
    callPopen("sed -i s$^env\.key_filename.*$env\.key_filename='{}'$ fabfile.py".format(keyDir + '/' + username))
    callPopen("sed -i s/^env\.roledefs.*/env.roledefs={{\\'{}\\':[\\'{}\\'],/ fabfile.py".format('stage',ip))

fabfile = os.path.join("C:\\Users\\jose_luis_guerrero\\Envs\\mylai\\prognos_calibration","fabfile.py")
    
if (platform.system() == "Windows"):    
    replace(fabfile, "^env\.hosts.*",         "env.hosts=['{}']".format(ip))
    replace(fabfile, "^env\.user.*",          "env.user='{}'".format(username))
    replace(fabfile, "^env\.key_filename.*",  "env.key_filename='{}'".format(os.path.join(keyDir,username)))
    replace(fabfile, "^env\.roledefs.*",      "env.roledefs={{'{}':['{}'],".format('stage',ip))

#Testing connection
#Adding key to remote machine
p = Popen("ssh -i {0}/{1} {1}@{2} -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no".format(keyDir,username,ip),shell=True,stdout=PIPE,stdin=PIPE)
print(p.communicate())

sed -i s/^env\.hosts.*/env.hosts=\['35.198.75.197']/ fabfile.py
sed -i s/^env\.user.*/env.user='jose-luis'/ fabfile.py
sed -i s$^env\.key_filename.*$env\.key_filename='/home/jose-luis/.ssh/gotmKeys/jose-luis'$ fabfile.py
sed -i s/^env\.roledefs.*/env.roledefs={\'stage\':[\'35.198.75.197\'],/ fabfile.py
(b'Linux gotm-langtjern 4.9.0-8-amd64 #1 SMP Debian 4.9.110-3+deb9u4 (2018-08-21) x86_64\n\nThe programs included with the Debian GNU/Linux system are free software;\nthe exact distribution terms for each program are described in the\nindividual files in /usr/share/doc/*/copyright.\n\nDebian GNU/Linux comes with ABSOLUTELY NO WARRANTY, to the extent\npermitted by applicable law.\n', None)


## Setting up GOTM and FABM in the remote machine

In [4]:
#ACPy .whl to be copied and installed on the remote machine
acpy_file = 'acpy-0.2-py2-none-any.whl'
schema_zip = 'GOTM_Schema.zip'
#callPopen('fab testConnection')
if not instanceExists:
    callPopen('fab getUtilities')
    callPopen('fab installACPy:{}'.format(acpy_file))
    callPopen('fab downloadModels') #note that the location of the key file is hardcoded in the getFABM function
    callPopen('fab upload:{},{}'.format(schema_zip,'~/code/schemas'))
    callPopen('fab compileModels')
    callPopen('fab getPROGNOS')
    callPopen('fab loadForcings')
callPopen('fab testRun:/home/{}/PROGNOS/langtjern/langtjern.xml'.format(username))

fab getUtilities
[35.198.75.197] Executing task 'getUtilities'
[35.198.75.197] Executing task 'update'
[35.198.75.197] Executing task 'updateMachine'
[35.198.75.197] run: sudo apt-get update
[35.198.75.197] out: 
[35.198.75.197] out: 0% [Working]
[35.198.75.197] out:             
[35.198.75.197] out: Ign:1 http://deb.debian.org/debian stretch InRelease
[35.198.75.197] out: 
[35.198.75.197] out: 0% [Waiting for headers] [Waiting for headers] [Connecting to packages.cloud.go
[35.198.75.197] out:                                                                                
[35.198.75.197] out: Get:2 http://security.debian.org stretch/updates InRelease [94.3 kB]
[35.198.75.197] out: 
[35.198.75.197] out: 0% [Waiting for headers] [2 InRelease 13.5 kB/94.3 kB 14%] [Connecting to packa
[35.198.75.197] out:                                                                                
[35.198.75.197] out: Get:3 http://deb.debian.org/debian stretch-updates InRelease [91.0 kB]
[35.198.75.197]

[35.198.75.197] out: 0% [11 Translation-en store 0 B] [Release.gpg gpgv 118 kB]
[35.198.75.197] out:                                                           
[35.198.75.197] out: Get:35 http://deb.debian.org/debian stretch-backports/main amd64 Packages 2018-09-23-0208.39.pdiff [588 B]
[35.198.75.197] out: 
[35.198.75.197] out: 0% [11 Translation-en store 0 B] [Release.gpg gpgv 118 kB] [35 Packages 588 B/5
[35.198.75.197] out:                                                                                
[35.198.75.197] out: 0% [11 Translation-en store 0 B] [Release.gpg gpgv 118 kB]
[35.198.75.197] out:                                                           
[35.198.75.197] out: Get:36 http://deb.debian.org/debian stretch-backports/main amd64 Packages 2018-09-23-2008.07.pdiff [235 B]
[35.198.75.197] out: 
[35.198.75.197] out: 0% [11 Translation-en store 0 B] [Release.gpg gpgv 118 kB] [36 Packages 235 B/2
[35.198.75.197] out:                                                         

[35.198.75.197] out: 0% [64 Packages rred 942 kB] [6 InRelease gpgv 6,377 B]
[35.198.75.197] out:                                                        
[35.198.75.197] out: 0% [64 Packages rred 2,023 kB]
[35.198.75.197] out: 0% [64 Packages rred 2,023 kB] [7 InRelease gpgv 3,843 B] [Waiting for headers]
[35.198.75.197] out:                                                                                
[35.198.75.197] out: Get:81 http://packages.cloud.google.com/apt cloud-sdk-stretch/main amd64 Packages [50.0 kB]
[35.198.75.197] out: 
[35.198.75.197] out: 0% [64 Packages rred 2,023 kB] [7 InRelease gpgv 3,843 B] [81 Packages 28.4 kB/
[35.198.75.197] out:                                                                                
[35.198.75.197] out: 0% [64 Packages rred 2,023 kB] [7 InRelease gpgv 3,843 B]
[35.198.75.197] out: 0% [64 Packages rred 2,023 kB] [81 Packages store 0 B] [7 InRelease gpgv 3,843 
[35.198.75.197] out:                                                       

[35.198.75.197] out: 13% [8 cmake-data 0 B/1,216 kB 0%] [7 linux-libc-dev 131 kB/1,351 kB 10%]
[35.198.75.197] out:                                                                          
[35.198.75.197] out: 15% [7 linux-libc-dev 1,049 kB/1,351 kB 78%]
[35.198.75.197] out:                                             
[35.198.75.197] out: Get:9 http://deb.debian.org/debian stretch/main amd64 liblzo2-2 amd64 2.08-1.2+b2 [55.0 kB]
[35.198.75.197] out: 
[35.198.75.197] out: 15% [9 liblzo2-2 0 B/55.0 kB 0%] [7 linux-libc-dev 1,114 kB/1,351 kB 82%]
[35.198.75.197] out:                                                                          
[35.198.75.197] out: 16% [7 linux-libc-dev 1,180 kB/1,351 kB 87%]
[35.198.75.197] out:                                             
[35.198.75.197] out: Get:10 http://deb.debian.org/debian stretch/main amd64 libicu57 amd64 57.1-6+deb9u2 [7,700 kB]
[35.198.75.197] out: 
[35.198.75.197] out: 16% [10 libicu57 0 B/7,700 kB 0%] [7 linux-libc-dev 1,180 kB/1

[35.198.75.197] out: Get:40 http://deb.debian.org/debian stretch/main amd64 libgcc-6-dev amd64 6.3.0-18+deb9u1 [2,296 kB]
[35.198.75.197] out: 
[35.198.75.197] out: 48% [40 libgcc-6-dev 38.5 kB/2,296 kB 2%]
[35.198.75.197] out:                                          
[35.198.75.197] out: 50% [Waiting for headers]
[35.198.75.197] out:                          
[35.198.75.197] out: Get:41 http://deb.debian.org/debian stretch/main amd64 gcc-6 amd64 6.3.0-18+deb9u1 [6,900 kB]
[35.198.75.197] out: 
[35.198.75.197] out: 50% [41 gcc-6 35.8 kB/6,900 kB 1%]
[35.198.75.197] out:                                   
[35.198.75.197] out: 57% [Waiting for headers]
[35.198.75.197] out:                          
[35.198.75.197] out: Get:42 http://deb.debian.org/debian stretch/main amd64 gcc amd64 4:6.3.0-4 [5,196 B]
[35.198.75.197] out: 
[35.198.75.197] out:                          
[35.198.75.197] out: Get:43 http://deb.debian.org/debian stretch/main amd64 libc-dev-bin amd64 2.24-11+deb9u3 [258 kB]

...

[35.198.75.197] out: Resolving deltas:  11% (136/1231)   
[35.198.75.197] out: Resolving deltas:  13% (166/1231)   
[35.198.75.197] out: Resolving deltas:  15% (186/1231)   
[35.198.75.197] out: Resolving deltas:  20% (249/1231)   
[35.198.75.197] out: Resolving deltas:  21% (260/1231)   
[35.198.75.197] out: Resolving deltas:  22% (271/1231)   
[35.198.75.197] out: Resolving deltas:  24% (298/1231)   
[35.198.75.197] out: Resolving deltas:  25% (309/1231)   
[35.198.75.197] out: Resolving deltas:  29% (358/1231)   
[35.198.75.197] out: Resolving deltas:  29% (366/1231)   
[35.198.75.197] out: Resolving deltas:  30% (370/1231)   
[35.198.75.197] out: Resolving deltas:  31% (382/1231)   
[35.198.75.197] out: Resolving deltas:  32% (394/1231)   
[35.198.75.197] out: Resolving deltas:  33% (407/1231)   
[35.198.75.197] out: Resolving deltas:  34% (419/1231)   
[35.198.75.197] out: Resolving deltas:  35% (438/1231)   
[35.198.75.197] out: Resolving deltas:  36% (448/1231)   
[35.198.75.197

[35.198.75.197] out: Checking out files:  93% (753/806)   
[35.198.75.197] out: Checking out files:  93% (755/806)   
[35.198.75.197] out: Checking out files:  94% (758/806)   
[35.198.75.197] out: Checking out files:  95% (766/806)   
[35.198.75.197] out: Checking out files:  96% (774/806)   
[35.198.75.197] out: Checking out files:  97% (782/806)   
[35.198.75.197] out: Checking out files:  98% (790/806)   
[35.198.75.197] out: Checking out files:  99% (798/806)   
[35.198.75.197] out: Checking out files: 100% (806/806)   
[35.198.75.197] out: Checking out files: 100% (806/806), done.
[35.198.75.197] out: 


Done.
Disconnecting from 35.198.75.197... done.
fab loadForcings
[35.198.75.197] Executing task 'loadForcings'
[35.198.75.197] Executing task 'putForcings'
[35.198.75.197] run:  sudo rm -r ~/PROGNOS/langtjern/processed_data
[35.198.75.197] put: /home/jose-luis/Envs/prognos_get_data_py3/notebook/results/temp.obs -> /home/jose-luis/PROGNOS/langtjern/results/temp.obs
[35.198.75.197]

[35.198.75.197] out:     init_turbulence
[35.198.75.197] out:         reading turbulence namelists..
[35.198.75.197] out:         done.
[35.198.75.197] out:         allocation memory..
[35.198.75.197] out:         allocation memory..
[35.198.75.197] out:         allocation memory..
[35.198.75.197] out:         done.
[35.198.75.197] out:          
[35.198.75.197] out:         --------------------------------------------------------
[35.198.75.197] out:         You are using the k-epsilon model
[35.198.75.197] out:         with the following properties:
[35.198.75.197] out:          
[35.198.75.197] out:             ce1                                  =   1.4399999999999999     
[35.198.75.197] out:             ce2                                  =   1.9199999999999999     
[35.198.75.197] out:             ce3minus                             = -0.74437894320570175     
[35.198.75.197] out:             ce3plus                              =   1.0000000000000000     
[35.198.75.197] out

[35.198.75.197] out:             phosphate_input_target_bfl  UNITS m-3*m/s  phosphate_input target variable bottom flux
[35.198.75.197] out:             total_carbon_at_interfaces_calculator_result  mmol m-3*m  total_carbon_at_interfaces_calculator result
[35.198.75.197] out:             total_phosphorus_at_interfaces_calculator_result  mmol m-3*m  total_phosphorus_at_interfaces_calculator result
[35.198.75.197] out:         Using Eulerian solver
[35.198.75.197] out:         Using euler_forward()
[35.198.75.197] out:     init_gotm_fabm_input
[35.198.75.197] out:     done
[35.198.75.197] out:         done.
[35.198.75.197] out:  ------------------------------------------------------------------------
[35.198.75.197] out:     saving initial conditions
[35.198.75.197] out:         Processing output category /state:
[35.198.75.197] out:           - temp
[35.198.75.197] out:           - salt
[35.198.75.197] out:           - u
[35.198.75.197] out:           - uo
[35.198.75.197] out:          

[35.198.75.197] out:           20 %
[35.198.75.197] out:           30 %
[35.198.75.197] out:           40 %
[35.198.75.197] out:           50 %
[35.198.75.197] out:           60 %
[35.198.75.197] out:           70 %
[35.198.75.197] out:           80 %
[35.198.75.197] out:           90 %
[35.198.75.197] out:          100 %
[35.198.75.197] out:  ------------------------------------------------------------------------
[35.198.75.197] out:     clean_up
[35.198.75.197] out:     clean_hypsograph
[35.198.75.197] out:         de-allocation hypsograph memory ...
[35.198.75.197] out:         done
[35.198.75.197] out:     clean_meanflow
[35.198.75.197] out:         de-allocation meanflow memory ...
[35.198.75.197] out:         done.
[35.198.75.197] out:     clean_turbulence
[35.198.75.197] out:         de-allocating turbulence memory ...
[35.198.75.197] out:         done.
[35.198.75.197] out:     clean_observations
[35.198.75.197] out:         de-allocate observation memory ...
[35.198.75.197] ou

## Selecting parameters to calibrate

### Selecting parameters to calibrate

In [5]:
parameterDict = {'shf_factor' : 
            {'file' : 'airsea.nml', 'namelist': 'airsea', 'minimum' : 0.5, 'maximum' : 1.5, 'cal' : True} }
parameterDict['swr_factor'] =  {'file' : 'airsea.nml', 'namelist': 'airsea', 'minimum' : 0.5, 'maximum' : 1.5, 'cal' : True} 
parameterDict['wind_factor'] = {'file' : 'airsea.nml', 'namelist': 'airsea', 'minimum' : 0.0, 'maximum' : 3.0, 'cal' : True} 
parameterDict['k_min'] = {'file' : 'gotmturb.nml', 'namelist': 'turb_param', 'minimum' : 1e-9, 'maximum' : 1e-5, 'logscale' : 'True', 'cal' : True} 
parameterDict['A'] = {'file' : 'obs.nml', 'namelist': 'extinct', 'minimum' : 0.0, 'maximum' : 1.0, 'cal' : True} 
parameterDict['g1'] = {'file' : 'obs.nml', 'namelist': 'extinct', 'minimum' : 0.1, 'maximum' : 3.0, 'cal' : True} 
parameterDict['g2'] = {'file' : 'obs.nml', 'namelist': 'extinct', 'minimum' : 1.0, 'maximum' : 30.0, 'cal' : True}
parameterDict['instances/dom/parameters/kOM2'] = {'file' : 'fabm.yaml', 'minimum' : 0., 'maximum' : 1.1, 'cal' : True} 
parameterDict['instances/dom/parameters/kOM1'] = {'file' : 'fabm.yaml', 'minimum' : 0., 'maximum' : 300., 'cal' : True} 
parameterDict['instances/dom/parameters/Km_O2'] = {'file' : 'fabm.yaml', 'minimum' : 0.1, 'maximum' : 20., 'cal' : True} 
parameterDict['instances/dom/parameters/Km_NO3'] = {'file' : 'fabm.yaml', 'minimum' : 1., 'maximum' : 10., 'cal' : True} 
parameterDict['instances/dom/parameters/Kin_O2'] = {'file' : 'fabm.yaml', 'minimum' : 1., 'maximum' : 10., 'cal' : True}
parameterDict['instances/dom/parameters/qy_DOM'] = {'file' : 'fabm.yaml', 'minimum' : 1E-5, 'maximum' : 1E-2, 'cal' : True, 'logscale' : True}
parameterDict['instances/dom/parameters/theta'] = {'file' : 'fabm.yaml', 'minimum' : 0.5, 'maximum' : 1.5, 'cal' : True}
parameterDict['instances/dom/parameters/k_floc'] = {'file' : 'fabm.yaml', 'minimum' : 1E-14, 'maximum' : 1.5, 'cal' : True} 
print(parameterDict)

{'shf_factor': {'file': 'airsea.nml', 'namelist': 'airsea', 'minimum': 0.5, 'maximum': 1.5, 'cal': True}, 'swr_factor': {'file': 'airsea.nml', 'namelist': 'airsea', 'minimum': 0.5, 'maximum': 1.5, 'cal': True}, 'wind_factor': {'file': 'airsea.nml', 'namelist': 'airsea', 'minimum': 0.0, 'maximum': 3.0, 'cal': True}, 'k_min': {'file': 'gotmturb.nml', 'namelist': 'turb_param', 'minimum': 1e-09, 'maximum': 1e-05, 'logscale': 'True', 'cal': True}, 'A': {'file': 'obs.nml', 'namelist': 'extinct', 'minimum': 0.0, 'maximum': 1.0, 'cal': True}, 'g1': {'file': 'obs.nml', 'namelist': 'extinct', 'minimum': 0.1, 'maximum': 3.0, 'cal': True}, 'g2': {'file': 'obs.nml', 'namelist': 'extinct', 'minimum': 1.0, 'maximum': 30.0, 'cal': True}, 'instances/dom/parameters/kOM2': {'file': 'fabm.yaml', 'minimum': 0.0, 'maximum': 1.1, 'cal': True}, 'instances/dom/parameters/kOM1': {'file': 'fabm.yaml', 'minimum': 0.0, 'maximum': 300.0, 'cal': True}, 'instances/dom/parameters/Km_O2': {'file': 'fabm.yaml', 'minimum

In [6]:
#Function to generate XML config file for ACPy

def generate_XML_config(filename,dummy):
    document = ET.Element('config')
    transports = ET.SubElement(document, 'transports')
    database = ET.SubElement(transports,'transport', type='sqlite',path='langtjern.db')
    executable = ET.SubElement(document, 'executable', path='/usr/bin/gotm')
    setup_path = ET.SubElement(document, 'setup', path='../')
    parameters = ET.SubElement(document, 'parameters')
    observations = ET.SubElement(document, 'observations')

    #Adding parameters to calibrate
    for key, value in parameterDict.items():
        print(key,value['cal'])
        if value['cal']:
            temp = value
            #temp.rename(columns={"min": "minimum", "max": "maximum"})
            temp.pop('cal',None)
            temp['minimum'] = str(temp['minimum'])
            temp['maximum'] = str(temp['maximum'])
            print(temp)
            ET.SubElement(parameters, 'parameter', attrib=temp, variable=key)
            

    #Adding observations to calibrate against
    ET.SubElement(observations, 'variable',
                  source="./temp.obs",
                  modelpath="output.nc",
                  modelvariable="temp",
                  spinupyears="0"
                 )

    #ET.SubElement(observations, 'variable',
    #              source="./cdom_out.obs",
    #              modelpath="output.nc",
    #              modelvariable="dom_DOMb",
    #              spinupyears="0"
    #             )


    et = ET.ElementTree(document)

    f = BytesIO()
    et.write(f, encoding='utf-8', xml_declaration=True) 
    bla =f.getvalue()

    with open(filename,'wb') as out: ## Open temporary file as bytes
        out.write(bla) 
        
    print(bla)    

In [7]:
xml_filename = 'config_acpy.xml'
def on_generate_xml_clicked(filename,dummy):
    for i in items_checkbox : 
        if i.value :
            parameterDict[i.description]['cal'] = True
        else :
            parameterDict[i.description]['cal'] = False
    generate_XML_config(filename,parameterDict)   
    #print("Button clicked.")

items_checkbox = [  widgets.Checkbox(
                    value=True,
                    description=key,
                    disabled=False,
                    layout=widgets.Layout(width='350px', grid_area = ('check_' + key).replace("/","_"), justify = 'end')
                )               
                for key, value in parameterDict.items()
                ]

items_range = [ widgets.FloatRangeSlider(
                    value=[float(value['minimum']), float(value['maximum'])],
                    min=float(value['minimum']),
                    max=float(value['maximum']),
                    step=(float(value['maximum']) - float(value['minimum'])) / 100.,
                    description='',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='.2f',
                    layout=widgets.Layout(width='auto', grid_area = ("range_" + key).replace("/","_") )
                )    
                for key, value in parameterDict.items()
                ]


items_range_min = [ widgets.FloatRangeSlider(
                    value=[float(value['minimum']), float(value['maximum'])],
                    min=float(value['minimum']),
                    max=float(value['maximum']),
                    step=(float(value['maximum']) - float(value['minimum'])) / 100.,
                    description='',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='.2f',
                    layout=widgets.Layout(width='auto', grid_area = ("range_" + key).replace("/","_") )
                )    
                for key, value in parameterDict.items()
                ]




header  = widgets.Button(description='Select parameters to calibrate',
                 layout=widgets.Layout(width='auto', grid_area='header'),
                 style=widgets.ButtonStyle(button_color='lightblue'))


footer  = widgets.Button(description='Generate XML',
                 layout=widgets.Layout(width='auto', grid_area='footer'),
                 style=widgets.ButtonStyle(button_color='olive'))

footer.on_click(partial(on_generate_xml_clicked,xml_filename))

boxChildren = [header,] + items_checkbox + items_range + [footer,]

template_areas = '"header header"'
for key,val in parameterDict.items():
    template_areas += '"check_{0} range_{0}"'.format(key).replace("/","_")

template_areas += '"footer footer"'
# print(template_areas)

widgets.GridBox(children=boxChildren,
        layout=widgets.Layout(
            width='100%',
            grid_template_rows='auto ' * (len(parameterDict)),
            grid_template_columns='40% 60%',
            grid_template_areas=template_areas
            )
            )
          

GridBox(children=(Button(description='Select parameters to calibrate', layout=Layout(grid_area='header', width…

In [8]:
callPopen('fab runACPy:{}'.format(xml_filename))

fab runACPy:config_acpy.xml
[35.198.75.197] Executing task 'runACPy'
[35.198.75.197] put: config_acpy.xml -> ./PROGNOS/langtjern/acpy/config_acpy.xml

Done.
Disconnecting from 35.198.75.197... done.
